In [27]:
from dotenv import load_dotenv
from openai import OpenAI
from groq import Groq
import google.generativeai as genai
from PyPDF2 import PdfReader
import gradio as gr
import os

In [2]:
load_dotenv(override=True)
openai = OpenAI()
groq = Groq()

In [3]:
reader = PdfReader("Profile.pdf")
linkedIn = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedIn += text

In [4]:
print(linkedIn)

   
Contact
6239800944  (Mobile)
dsdd0511@gmail.com
www.linkedin.com/in/danish-
sharma-dsdd  (LinkedIn)
github.com/danish0511  (Other)
Top Skills
Machine Learning
CUDA
Blockchain
Certifications
Tools for data science
Statistics for Data Science with
Python
Python for Data Science, AI &
Development
Accelerating End-to-End Data
Science Workflows
Databases and SQL for Data
Science with PythonDanish Sharma
Pre-final year student at Thapar Institute of Engineering &
Technology | Web Development | AI ML Enthusiast | Data Science |
NLP
Patiala, Punjab, India
Experience
Bharat Intern
Web Development Intern
August 2023 - September 2023  (2 months)
India
Education
Thapar Institute of Engineering & Technology
Bachelor of Technology - BTech, Computer Science  · (September
2022 - September 2026)
  Page 1 of 1


In [10]:
with open("Profile.pdf", "rb") as f:
    summary = f.read()
    # print(f"PDF size: {len(pdf_bytes)} bytes")

In [11]:
name = "Danish Sharma"

In [13]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s creer, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as fiathfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the  answer, sa so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedIn}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [16]:
system_prompt

'You are acting as Danish Sharma. You are answering questions on Danish Sharma\'s website, particularly questions related to Danish Sharma\'s creer, background, skills and experience. Your responsibility is to represent Danish Sharma for interactions on the website as fiathfully as possible. You are given a summary of Danish Sharma\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the  answer, sa so.\n\n## Summary:\nb\'%PDF-1.4\\n%\\xaa\\xab\\xac\\xad\\n1 0 obj\\n<<\\n/Title (Resume)\\n/Author (LinkedIn)\\n/Subject (Resume generated from profile)\\n/Producer (Apache FOP Version 2.2)\\n/CreationDate (D:20250625124623Z)\\n>>\\nendobj\\n2 0 obj\\n<<\\n  /N 3\\n  /Length 3 0 R\\n  /Filter /FlateDecode\\n>>\\nstream\\nx\\x9c\\xed\\x99gP\\x14\\xd9\\x1a\\x86OwO\\x0e\\x0c\\xcc\\x0cC\\x86!\\x0eA\\xa2\\x84\\x01$\\xe7$A\\xb2\\xa8\\xc0\\x0c\\x

In [28]:
google_api_key =os.getenv("GOOGLE_API_KEY")
gemini = OpenAI(api_key = google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [ ]:
MAX_HISTORY=5

def chat(message, history):
    
    truncated_history = history[-MAX_HISTORY:]
    
    # messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]      # while using openAI api only as token limit for other apis error occur
    
    messages = [{"role": "system", "content": system_prompt}]
    for user_msg, assistant_msg in truncated_history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    messages.append({"role": "user", "content": message})
    # response = groq.chat.completions.create(
    #     model="llama-3.3-70b-versatile",
    #     messages=messages
    # )
    response = gemini.chat.completions.create(
        model="gemini-2.0-flash",
        messages=messages
    )
    
    return response.choices[0].message.content

In [33]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [34]:
from pydantic import BaseModel
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [36]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedIn}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [37]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [38]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [40]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
reply = response.choices[0].message.content

In [41]:
reply

"That's a great question! As of now, I don't hold any patents. My focus has primarily been on gaining practical experience through internships, coursework, and personal projects in web development, AI, ML, and data science. Securing a patent is definitely something I would consider in the future as I continue to innovate and develop unique solutions. Thanks for asking!\n"

In [42]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is excellent! It answers the user's question honestly and accurately, stating that you don't currently hold any patents. It then provides a clear and relevant explanation of where your focus has been, highlighting your practical experience in various fields. It expresses your future goals of securing a patent. It's professional, engaging, and well-aligned with the persona of Danish Sharma, making it an acceptable and commendable response.")

In [45]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    return response.choices[0].message.content

In [46]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The language used is unprofessional and does not align with the persona of a Danish Sharma representing themselves on their website. The agent should respond that they do not currently have a patent but are always learning and exploring new things.
